In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from vnstock import *
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import yfinance as yf
from math import sqrt
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [13]:
data_ticker = pd.read_excel("C:\\Users\\Chi Bao\\OneDrive\\Desktop\\Đồ án tốt nghiệp\\data_ticker.xlsx")

FI = ['GC=F','SI=F','CL=F','^GSPC','^DJI','^IXIC','^RUT','^FTSE','^N225','^NYA']
df = pd.DataFrame()
for symbol in FI:
    data = yf.download(symbol, start='2012-03-20', end='2023-09-06')
    df[symbol] = data['Adj Close']

column_mapping = {
    'GC=F': 'Gold',
    'SI=F': 'Silver',
    'CL=F': 'Crude Oil',
    '^GSPC': 'S&P 500',
    '^DJI': 'Dow Jones',
    '^IXIC': 'NASDAQ',
    '^RUT': 'Russell 2000',
    '^FTSE': 'FTSE 100',
    '^N225': 'Nikkei 225',
    '^NYA': 'NYSE'
}

df = df.rename(columns=column_mapping)
df.dropna()

returns_df = df.pct_change()*100
returns_df.columns = [col + '_return' for col in returns_df.columns]
returns_df = returns_df.dropna()

###

FI = ['VNINDEX','HNX','VN30','HNX30','UPCOM']
data = stock_historical_data('VNINDEX', '2012-03-19', '2023-09-05', "1D", "index")
d_new = data.set_index('time')
d_new['VNINDEX'] = data['close']
df1 = pd.DataFrame()
for symbol in FI:
    data = stock_historical_data(symbol, '2012-03-19', '2023-09-05', "1D", "index")
    data = data.set_index('time')
    df1[symbol] = data['close']
df1 = df1.dropna()

returns_df1 = df1.pct_change()*100
returns_df1.columns = [col + '_return' for col in returns_df1.columns]
returns_df1 = returns_df1.dropna()
returns_df1.head(42069)

###

lagreturn = pd.DataFrame()

for column in returns_df.columns:
    lagreturn[f'{column}_Lag1'] = returns_df[column].shift(1)
for column in returns_df.columns:
    lagreturn[f'{column}_Lag2'] = returns_df[column].shift(2)
lagreturn = lagreturn.dropna()

lagreturn1 = pd.DataFrame()
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag1'] = returns_df1[column].shift(1)
for column in returns_df1.columns:
    lagreturn1[f'{column}_Lag2'] = returns_df1[column].shift(2)
lagreturn1 = lagreturn1.dropna()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [14]:
def do_tre_vnstock(symbol, start_date, end_date, lags=10): 
    ts = stock_historical_data(symbol,start_date, end_date)
    ts = ts.dropna(subset=['close'])


    tsret = pd.DataFrame(index=ts.index)
    tsret['time']=ts['time']
    tsret["Today"] = ts["close"].pct_change()*100

    for i,x in enumerate(tsret["Today"]):
        if (abs(x) < 0.0001):
            tsret["Today"][i] = 0.0001

    for i in range(0, lags):
        tsret["Lag%s" % str(i+1)] = tsret["Today"].shift(i+1)

    tsret["Direction"] = np.sign(tsret["Today"])
    tsret = tsret.dropna()
    tsret = tsret.set_index('time')
    bpt = pd.merge(tsret, lagreturn, left_index=True, right_index=True)
    bpt1 = pd.merge(bpt, lagreturn1, left_index=True, right_index=True)
    bpt1.fillna(0, inplace=True)
    bpt1 = bpt1[(bpt1 != 0).all(axis=1)]
    return bpt1

In [15]:
#MODEL
list_i = []
list_return_price = []
list_rmse, list_mse, list_mae = [], [], []
list_k = []
list_t = []
list_r2_train = []
list_r2_test = []

for i in data_ticker['ticker']:
  try:
    data = do_tre_vnstock(i, '2012-03-19', '2023-09-05', lags=10)

    X = data.loc[:, ~data.columns.isin(['Today'])]
    y = data["Today"]

    list_k1 = []
    list_t1 = []
    list_score = []

    x = round(len(X)*0.1)
    for t in range(2,x+1):
        for k in np.arange(0.1, 0.5, 0.01):
            model = KNeighborsRegressor(n_neighbors=t)  
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=k, random_state=12, shuffle=False)
            model.fit(X_train, y_train)
            yhat_test = model.predict(X_test)
            score = model.score(X_test, y_test)
            list_k1.append(k)
            list_t1.append(t)
            list_score.append(score)

        score_max = max(list_score)
        a = list_score.index(score_max)
        k = list_k1[a]
        t = list_t1[a]

        X_train, X_test,y_train, y_test=train_test_split(X,y,test_size=k,random_state=12, shuffle=False)
        model.fit(X_train,y_train)
        yhat_test = model.predict(X_test)
        r2_train = model.score(X_train, y_train)
        r2_test = model.score(X_test, y_test)
        mse = mean_squared_error(y_test, yhat_test, squared=True)
        rmse = mean_squared_error(y_test, yhat_test, squared=False)
        mae = mean_absolute_error(y_test, yhat_test)

        X_predict = [X.iloc[-1].values.tolist()]
        Y_predict = model.predict(X_predict)

    list_i.append(i)
    list_k.append(k)
    list_t.append(t)
    list_return_price.append(Y_predict)
    list_r2_train.append(r2_train)
    list_r2_test.append(r2_test)
    list_rmse.append(rmse)
    list_mse.append(mse)
    list_mae.append(mae)

    print('Stock:',i,f' Predict: {Y_predict.item():.2f}',f' R2_train: {r2_train:.2f}',f' R2_test: {r2_test:.2f}',
          f' RMSE: {rmse:.2f}',f' MSE: {mse:.2f}', f'MAE: {mae:.2f}',f' test size: {k:.2f}',f'neighbors: {t}')
  except Exception as e:
    print(f"Stock: {i} -Unpredictable!\n" f'{e}')
    continue

Stock: SSI  Predict: -0.04  R2_train: 0.06  R2_test: 0.03  RMSE: 2.76  MSE: 7.64 MAE: 2.03  test size: 0.35 neighbors: 18
Stock: BCM  Predict: -0.00  R2_train: 0.06  R2_test: 0.05  RMSE: 1.31  MSE: 1.73 MAE: 0.87  test size: 0.11 neighbors: 9
Stock: VHM  Predict: -0.05  R2_train: 0.09  R2_test: 0.08  RMSE: 1.94  MSE: 3.74 MAE: 1.38  test size: 0.13 neighbors: 23
Stock: VIC  Predict: -0.02  R2_train: 0.06  R2_test: 0.05  RMSE: 2.08  MSE: 4.32 MAE: 1.24  test size: 0.23 neighbors: 10
Stock: VRE  Predict: 0.09  R2_train: 0.09  R2_test: 0.14  RMSE: 1.46  MSE: 2.12 MAE: 1.09  test size: 0.12 neighbors: 14
Stock: BVH  Predict: 0.25  R2_train: 0.10  R2_test: 0.20  RMSE: 1.04  MSE: 1.07 MAE: 0.73  test size: 0.12 neighbors: 23
Stock: POW  Predict: 0.33  R2_train: 0.08  R2_test: 0.17  RMSE: 1.33  MSE: 1.77 MAE: 0.90  test size: 0.10 neighbors: 18
Stock: GAS  Predict: 0.21  R2_train: 0.10  R2_test: 0.18  RMSE: 0.97  MSE: 0.94 MAE: 0.73  test size: 0.20 neighbors: 17
Stock: ACB  Predict: 0.24  R2

In [16]:
KQ = pd.DataFrame({'Stock': list_i, 'Predict': list_return_price, 'R2_train': list_r2_train,'R2_test': list_r2_test,
                    'RMSE': list_mse,'MSE': list_rmse, 'MAE': list_mae,'test size': list_k, 'neighbors': list_t})
KQ.to_excel("Kết quả linh hoạt test size.xlsx")
KQ.head(42069)

,Stock,Predict,R2_train,R2_test,RMSE,MSE,MAE,test size,neighbors
0,SSI,[-0.04424811643037758],0.064193,0.030400,7.640935,2.764224,2.030579,0.35,18
1,BCM,[-0.0035207985589991997],0.057334,0.051302,1.726511,1.313968,0.866836,0.11,9
2,VHM,[-0.05394360808704153],0.086479,0.078679,3.744253,1.935007,1.379719,0.13,23
3,VIC,[-0.024529924389810262],0.063733,0.052537,4.319171,2.078262,1.239583,0.23,10
4,VRE,[0.0919332411249334],0.086424,0.138151,2.121012,1.456370,1.085075,0.12,14
...,...,...,...,...,...,...,...,...,...
389,TDP,[-0.027233908481202863],0.063328,0.024801,8.204294,2.864314,1.966814,0.10,6
390,NO1,[-0.555629905354687],0.177401,0.043423,8.793204,2.965334,2.131897,0.35,13
391,L10,[1.1838774370027718],0.078022,0.034657,35.766345,5.980497,4.975268,0.18,10
392,REE,[0.06088207928985495],0.077114,0.080006,1.669373,1.292042,0.945923,0.13,10
